In [1]:
'''
 Yacine Mahdid 2019-11-02
 This script was created for the analysis of the graph features for consciousness decoding
 The goal here is to take the CSV created in the preprocess.m file and to manipulate it to get labels, participant id and such.
 This script was tested with atruncated version of the full dataset (only alpha, only ec1 and only 1 null network) while the real dataset was getting generated

'''

'\n Yacine Mahdid 2019-11-02\n This script was created for the analysis of the graph features for consciousness decoding\n The goal here is to take the CSV created in the preprocess.m file and to manipulate it to get labels, participant id and such.\n This script was tested with atruncated version of the full dataset (only alpha, only ec1 and only 1 null network) while the real dataset was getting generated\n\n'

In [9]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

# First step will be to import the feature data and the labels
data_path = "C:/Users/biapt/Documents/wei_data_fixed_aec.csv";
data = pd.read_csv(data_path)
# here what everything means:
'''
frequencies = {"alpha","beta","delta","theta","gamma"};
epochs = {'ec1','emf5','ec1','if5','emf5','eml5','ec3','ec8'};
graph = {"aec","pli"}
The ids we see are mapping to these
'''
data

,p_id,frequency,epoch,graph,window,mean_1,mean_2,mean_3,mean_4,mean_5,...,std_74,std_75,std_76,std_77,std_78,std_79,std_80,std_81,std_82,community
0,1,1,1,0,1,0.214410,0.206480,0.186960,0.095920,0.185300,...,0.163960,0.157790,0.149060,0.157530,0.130100,0.107980,0.129380,0.184910,0.133150,0.143490
1,1,1,1,1,1,0.135180,0.133150,0.066425,0.142670,0.187410,...,0.109020,0.109170,0.107650,0.102930,0.135180,0.100110,0.115610,0.088783,0.104130,0.116620
2,1,1,1,0,2,0.173520,0.176050,0.207510,0.196660,0.100010,...,0.125690,0.104380,0.137390,0.131190,0.127780,0.099890,0.119320,0.118170,0.103930,0.081770
3,1,1,1,1,2,0.135480,0.110920,0.099013,0.136760,0.166900,...,0.137890,0.120680,0.102300,0.103010,0.117330,0.141430,0.098518,0.077446,0.136830,0.126540
4,1,1,1,0,3,0.090807,0.194400,0.160160,0.122470,0.049862,...,0.142590,0.139910,0.132330,0.131750,0.148870,0.091402,0.151970,0.134850,0.146600,0.147540
5,1,1,1,1,3,0.089552,0.056574,0.085036,0.109230,0.130940,...,0.125230,0.121790,0.114710,0.109170,0.103370,0.097368,0.077802,0.074077,0.110460,0.109300
6,1,1,1,0,4,0.092985,0.167780,0.171060,0.174230,0.075009,...,0.144860,0.141310,0.200430,0.200280,0.164220,0.109050,0.116630,0.113940,0.140220,0.140750
7,1,1,1,1,4,0.056143,0.058897,0.039806,0.064573,0.101440,...,0.110750,0.108390,0.105170,0.127820,0.156570,0.069891,0.062892,0.078847,0.105680,0.115430
8,1,1,1,0,5,0.085757,0.135000,0.094392,0.174200,0.048321,...,0.113320,0.126320,0.123960,0.128090,0.130890,0.144300,0.140410,0.124030,0.115860,0.139690
9,1,1,1,1,5,0.104530,0.089392,0.081475,0.106140,0.129180,...,0.103350,0.082421,0.070377,0.096452,0.135200,0.072682,0.095998,0.080907,0.119650,0.118070


In [11]:
# We will keep the same structure than we did in the AEC vs wPLI project for the machine learning

# Machine Learning 
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Sklearn utils
from sklearn.base import clone
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

# aec = 0 pli = 1
analysis_type = 0
epochs_to_keep = [1,2]
analysis = data['graph'] == analysis_type
baseline = data['epoch'] == epochs_to_keep[0]
other = data['epoch'] == epochs_to_keep[1]
subset_data = data[analysis & (baseline | other)]

clfs = [LinearDiscriminantAnalysis(solver='svd'), SVC(kernel='linear', C=0.1),SVC(kernel='linear', C=0.5), SVC(kernel='linear', C=1.0), SVC(kernel='rbf', C=0.1), SVC(kernel='rbf',C=1.0)]
selected_clf = clfs[3]

def classify(dataset, original_clf):
    clf = clone(original_clf)
    # Initialize the Result data structures
    cms = []
    accuracies = []
    reports = []
    # TODO: Check in the MATLAB file how to not have to do this + 1
    for test_id in range(1,10):
        print("Participant: " + str(test_id) + " in hold-out set:")
        
        # Split the data in a leave one subject out manner
        
        # Get the training and test dataset
        training_dataset = dataset[dataset['p_id'] != test_id]
        test_dataset = dataset[dataset['p_id'] == test_id]
        
        # Get the X and Y 
        X_train = training_dataset.iloc[:,5:]
        y_train = training_dataset['epoch']
        
        # Training the min max normalizer
        min_max_scaler = MinMaxScaler()
        min_max_scaler.fit(X_train)
        X_train = min_max_scaler.transform(X_train)
        
        X_test = test_dataset.iloc[:,5:]
        y_test = test_dataset['epoch']
        
        # Normalize the test set using a training scaler
        X_test = min_max_scaler.transform(X_test)
        
        # Fitting our model
        clf.fit(X_train, y_train)

        # predicting
        y_pred = clf.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)

        report = classification_report(y_test, y_pred, output_dict=True)
        accuracy = accuracy_score(y_test, y_pred)
        print("Generalization accuracy: " + str(accuracy))
        print(cm)
        print(report)
        
        # Saving the results
        cms.append(cm)
        accuracies.append(accuracy)
        reports.append(report)
        
    return (cms,accuracies,reports)

(cms, accuracies, report) = classify(subset_data, selected_clf)
print("Mean accuracy is: " + str(np.mean(accuracies)))

Participant: 1 in hold-out set:
Generalization accuracy: 0.9130434782608695
[[51  6]
 [ 4 54]]
{'1': {'precision': 0.9272727272727272, 'recall': 0.8947368421052632, 'f1-score': 0.9107142857142856, 'support': 57}, '2': {'precision': 0.9, 'recall': 0.9310344827586207, 'f1-score': 0.9152542372881356, 'support': 58}, 'accuracy': 0.9130434782608695, 'macro avg': {'precision': 0.9136363636363636, 'recall': 0.9128856624319419, 'f1-score': 0.9129842615012106, 'support': 115}, 'weighted avg': {'precision': 0.9135177865612649, 'recall': 0.9130434782608695, 'f1-score': 0.9130040004210969, 'support': 115}}
Participant: 2 in hold-out set:
Generalization accuracy: 0.8782608695652174
[[57  0]
 [14 44]]
{'1': {'precision': 0.8028169014084507, 'recall': 1.0, 'f1-score': 0.890625, 'support': 57}, '2': {'precision': 1.0, 'recall': 0.7586206896551724, 'f1-score': 0.8627450980392156, 'support': 58}, 'accuracy': 0.8782608695652174, 'macro avg': {'precision': 0.9014084507042254, 'recall': 0.8793103448275862,